In [1]:
raw_dat_dir = '/home/grace/Documents/python/ttc_inhalation/data/raw/'
processed_dat_dir = '/home/grace/Documents/python/ttc_inhalation/data/processed/'
interim_dat_dir = '/home/grace/Documents/python/ttc_inhalation/data/interim/'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import os
import glob

In [4]:
for f in os.listdir(raw_dat_dir):
    print(f)

Original_ToxVal_v7_24-04-2019.RData
Original_ToxVal_v7_24-04-2019.csv
.ipynb_checkpoints
.gitkeep
ToxVal_v7.csv


In [5]:
def loadTXdata(file, dat_dir = raw_dat_dir):
    df = pd.read_csv(dat_dir+file)
    return  df

In [6]:
toxval = loadTXdata('Original_ToxVal_v7_24-04-2019.csv')
toxval.head()


/home/grace/anaconda3/envs/clone_newchem/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (34,51,52,64) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,chemical_id,toxval_id,toxval_uuid,parent_toxval_uuid,chemical_id_original,species_id,study_id,quality_id,priority_id,manual_qc_flag,...,details_text,datestamp,lifestage,generation,casrn,name,dsstox_substance_id,dsstox_compound_id,chemical_id_external,qsar_ready_smiles
0,147,1,695c5d4cd01afc10a34ad8a64f012bc1,-,147,24967,-1,-1,1,-,...,ATSDR Details,2019-04-22 16:58:52,-,-,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,-,CCC1=CC=CC=C1
1,147,2,aa3416372e86e2ad70eb36a37835fd88,-,147,24967,-1,-1,1,-,...,ATSDR Details,2019-04-22 16:58:52,-,-,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,-,CCC1=CC=CC=C1
2,147,3,9f8f21b321fffc119d6596113e6a42ad,-,147,24967,-1,-1,1,-,...,ATSDR Details,2019-04-22 16:58:52,-,-,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,-,CCC1=CC=CC=C1
3,147,4,f070a0caa6f198681d6eb29044052a03,-,147,24967,-1,-1,1,-,...,ATSDR Details,2019-04-22 16:58:52,-,-,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,-,CCC1=CC=CC=C1
4,147,5,c76087de396ffd785ff941c64bf41961,-,147,24967,-1,-1,1,-,...,ATSDR Details,2019-04-22 16:58:52,-,-,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,-,CCC1=CC=CC=C1


In [20]:
toxval[toxval['name'] == 'Ethylbenzene'].shape

(389, 50)

In [8]:
toxval.columns.tolist()

cols_to_drop = ['chemical_id',
 'toxval_id','toxval_uuid','parent_toxval_uuid','species_id',
 'study_id','priority_id','manual_qc_flag','source_url','subsource_url','record_url','source_source_id','toxval_numeric_converted','toxval_units_converted','chemical_id_external']

In [9]:
#len(cols_to_drop)

In [10]:
toxval.drop(cols_to_drop, axis = 1, inplace = True)

In [11]:
toxval.shape

(675694, 50)

In [12]:
mycols = toxval.columns.tolist()

In [13]:
mycols2 = mycols[45:] + mycols[:45]

In [14]:
#mycols2

In [15]:
toxval = toxval[mycols2]

In [21]:
#toxval[toxval.name == 'Ethylbenzene']

In [22]:
toxval.risk_assessment_class.unique()

array(['subacute', 'chronic', 'acute', 'repeat dose', 'subchronic',
       'cancer', 'short-term', 'subacute, subchronic',
       'ecotoxicity invertebrate', 'reproductive developmental',
       'ecotoxicity plants', 'reproductive', 'in vitro',
       'genotoxicity, repeat dose', 'developmental',
       'subacute, neurotoxicity', 'multigenerational reproductive',
       'cancer, repeat dose', 'neurotoxicity', 'chronic, subchronic',
       'air quality', 'water quality', 'clinical',
       'developmental neurotoxicity', 'human', 'chemcial property',
       'genotoxicity', 'special toxicity study', 'Immunotoxicology'],
      dtype=object)

In [23]:
study_length = []
for x, y, z, a in list(zip(toxval['risk_assessment_class'], toxval['study_type'], toxval['study_duration_value'], toxval['study_duration_units'])):
    if (x in ['short-term','repeat dose'] and y == 'chronic'):
        study_length.append('chronic')
    elif (x in ['short-term','repeat dose']) and (y not in ["reproductive", "subchronic"]) and (z >=100) and (a == "day"):
        study_length.append('chronic')
    elif (x in ["short-term", "repeat dose"]) and (y not in ["reproductive", "subchronic"]) and (z >=14) and (a == "week"):
        study_length.append('chronic')
    elif (x in ["short-term", "repeat dose"]) and (y not in ["reproductive", "subchronic"]) and (z>=4) and (a == "month"):
        study_length.append('chronic')
    elif (x in ["short-term", "repeat dose"]) and (y not in ["reproductive", "subchronic"]) and (z >=1) and (a == "year"):
        study_length.append('chronic')
    elif (x in ["short-term", "repeat dose"]) and  (y == "subchronic"):
        study_length.append('subchronic')
    elif (x in ['short-term','repeat dose']) and (y not in ["reproductive", "chronic"]) and (z>=30) and  (z <100) and (a == "day"):
        study_length.append('subchronic')
    elif (x in ['short-term','repeat dose']) and (y not in ["reproductive", "chronic"]) and (z>=5) and  (z <14) and (a == "week"):
        study_length.append('subchronic')
    elif (x in ['short-term','repeat dose']) and (y not in ["reproductive", "chronic"]) and (z>=1.5) and  (z <4) and (a == "month"):
        study_length.append('subchronic')
    elif (x == 'repeat dose') and (y not in ["acute", "reproductive", "chronic"]) and (z==-1):
        study_length.append('subchronic')
    elif (x == 'repeat dose') and (y not in ["acute", "reproductive", "chronic"]) and (z==0):
        study_length.append('subchronic')
    elif (x in ['short-term','repeat dose']) and (y == "reproductive"):
        study_length.append('reproductive')
    else:
        study_length.append(x)
        

toxval['study_length'] = study_length

In [24]:
len(study_length)

675694

### Adding filters for study types and PODs and routes of exposure

In [25]:
mask = (toxval['exposure_route'] == 'inhalation') & (toxval['toxval_type'].isin(['NOAEL','NOEL','NOAEC', 'NOEC'])) & (toxval['study_length'].isin(['subchronic','chronic','reproductive developmental','reproductive','developmental','reproductive developmental','multigenerational reproductive']))

In [26]:
toxval_filt = toxval[mask]

In [27]:
toxval_filt.qsar_ready_smiles.nunique()

697

In [28]:
toxval_filt.dsstox_substance_id.nunique()

1594

In [29]:
#toxval_filt.species_original.unique()

mask2 = toxval_filt['species_original'].isin(['rat','rabbit','mouse', 'rats','mouse / rat','other: rabbits, rats and mice', 'other: rats, mice, rabbits','rat / mouse','Rabbit','Mouse','other: rats and mice','other: rat and mice','other: mice, rats, and rabbits','other: rat and mouse'])
toxval_filt2 = toxval_filt[mask2]
toxval_filt2.shape

(2057, 51)

In [30]:
toxval_filt2.dsstox_substance_id.nunique()

533

### Going back to the ToxVal dataset pre-cleaned but with the study type to verify consistency with the processed ToxValv7 from Nelms

In [31]:
toxval.head()
toxval.shape

(675694, 51)

In [32]:
toxval.columns.tolist()

toxval.rename(columns = {'dsstox_substance_id': 'DTXSID', 'species_original': 'species_common'}, inplace = True)

In [33]:
toxval.species_common.unique().tolist()

toxval['species_common'] = [e.lower() for e in toxval['species_common']]

In [34]:
#[e for e in toxval['species_common'] if 'rat' in e]

In [35]:
#import re
#regex=re.compile(".*rat.*")
#set([m.group(0) for l in toxval['species_common'] for m in [regex.search(l)] if m])

In [36]:
toxval.columns

Index(['casrn', 'name', 'DTXSID', 'dsstox_compound_id', 'qsar_ready_smiles',
       'chemical_id_original', 'quality_id', 'source', 'subsource',
       'toxval_type', 'toxval_type_original', 'toxval_subtype',
       'toxval_subtype_original', 'toxval_numeric', 'toxval_numeric_original',
       'toxval_numeric_qualifier', 'toxval_numeric_qualifier_original',
       'toxval_units', 'toxval_units_original', 'risk_assessment_class',
       'study_type', 'study_type_original', 'study_duration_class',
       'study_duration_class_original', 'study_duration_value',
       'study_duration_value_original', 'study_duration_units',
       'study_duration_units_original', 'species_common', 'strain',
       'strain_original', 'sex', 'sex_original', 'population',
       'exposure_route', 'exposure_route_original', 'exposure_method',
       'exposure_method_original', 'exposure_form', 'exposure_form_original',
       'media', 'media_original', 'year', 'year_original', 'human_eco',
       'critical_ef

In [37]:
toxval7 = pd.read_csv(raw_dat_dir+'ToxVal_v7.csv')

/home/grace/anaconda3/envs/clone_newchem/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
toxval7.shape

(675694, 34)

### Checking if toxval7 and toxval are the same as far as study length

In [39]:
toxval7['study_length'].value_counts(dropna = False)

subacute                          180129
acute                             173829
chronic                           114502
subchronic                         54861
reproductive                       42711
ecotoxicity invertebrate           38178
short-term                         26730
developmental                      10563
cancer                              7222
repeat dose                         6001
genotoxicity                        4205
ecotoxicity plants                  3892
multigenerational reproductive      3471
chemcial property                   2862
water quality                       1854
neurotoxicity                       1689
reproductive developmental          1187
developmental neurotoxicity          745
special toxicity study               421
air quality                          345
human                                142
clinical                             119
Immunotoxicology                      24
subacute, subchronic                   4
cancer, repeat d

In [40]:
toxval['que'] = np.where((toxval['study_length'] == toxval7['study_length']),toxval['study_length'], np.nan)

In [41]:
toxval['que'].value_counts(dropna = False)

subacute                          180129
acute                             173829
chronic                           114502
subchronic                         54861
reproductive                       42711
ecotoxicity invertebrate           38178
short-term                         26730
developmental                      10563
cancer                              7222
repeat dose                         6001
genotoxicity                        4205
ecotoxicity plants                  3892
multigenerational reproductive      3471
chemcial property                   2862
water quality                       1854
neurotoxicity                       1689
reproductive developmental          1187
developmental neurotoxicity          745
special toxicity study               421
air quality                          345
human                                142
clinical                             119
Immunotoxicology                      24
subacute, subchronic                   4
cancer, repeat d

In [43]:
#toxval.to_csv(processed_dat_dir+'ToxVal_wstudylength.csv')